In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')
from features_creation import cate_colName
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold,cross_validate,GridSearchCV
from catboost import CatBoostClassifier
from features_creation import *
import hyperopt
from hyperopt import fmin,tpe,Trials,partial,hp
from hyperopt.early_stop import no_progress_loss
import lightgbm
from xgboost import XGBClassifier
import gc
from sklearn.feature_selection import RFECV

In [2]:
application_train = pd.read_csv('/root/autodl-tmp/now/down_data/application_train.csv')
application_test = pd.read_csv('/root/autodl-tmp/now/down_data/application_test.csv')
bureau = pd.read_csv('/root/autodl-tmp/now/down_data/bureau.csv')
bureau_balance = pd.read_csv('/root/autodl-tmp/now/down_data/bureau_balance.csv')
POS_CASH_balance = pd.read_csv('/root/autodl-tmp/now/down_data/POS_CASH_balance.csv')
credit_card_balance = pd.read_csv('/root/autodl-tmp/now/down_data/credit_card_balance.csv')
previous_application = pd.read_csv('/root/autodl-tmp/now/down_data/previous_application.csv')
installments_payments = pd.read_csv('/root/autodl-tmp/now/down_data/installments_payments.csv')

In [3]:
#独热编码
def OneHot_data(df,concat_df=False):
    #选取离散变量
    columns = df.select_dtypes('object').columns.tolist() + df.select_dtypes('bool').columns.tolist()
    oh = preprocessing.OneHotEncoder(drop='if_binary')
    oh.fit_transform(df[columns])
    hot_data = pd.DataFrame(oh.fit_transform(df[columns]).toarray(),columns=cate_colName(oh,columns))  #衍生出的独热编码数据
    if concat_df:  #如果concat_df为True,将原特征和编码特征融合
        new_data = pd.concat([df.drop(columns=df[columns],axis=1),hot_data],axis=1)
        return new_data,cate_colName(oh,columns)
    else:
        return hot_data,cate_colName(oh,columns)

#哑变量
def Label_Encoder(df,concat_df=False):
    columns = df.select_dtypes('object').columns.tolist()
    le = preprocessing.OrdinalEncoder()
    le_data = pd.DataFrame(le.fit_transform(df[columns]),columns=columns)
    if concat_df:  #如果concat_df为True,将原特征和编码特征融合
        new_data = pd.concat([df.drop(columns=df[columns],axis=1),le_data],axis=1)
        return new_data,columns
    else:
        return le_data,columns

In [4]:
def Submissing(proba,file_name):
    submission = pd.read_csv('/root/autodl-tmp/now/down_data/sample_submission.csv')
    submission['TARGET']  = proba
    submission.to_csv('/root/autodl-tmp/now/submission/{}.csv'.format(file_name),index=False)

In [5]:
import re
#删除列名里的特殊字符
def rename_columns(df):
    new_columns = []
    for col in df.columns:
        new_col = re.sub('[^0-9a-zA-Z]+', '_', col)  # 用下划线替换特殊字符
        new_columns.append(new_col)
    df.columns = new_columns
    return df

In [6]:
def Test_score_function(application_train_and_test,model,file_name,test=False):
    # reduce_mem_usage(application_train_and_test)
    
    #选出离散特征和连续特征
    category_cols = application_train_and_test.iloc[:,2:].select_dtypes('object').columns.tolist() + application_train_and_test.iloc[:,2:].select_dtypes('bool').columns.tolist()
    numeric_cols = application_train_and_test.iloc[:,2:].select_dtypes('int').columns.tolist() + application_train_and_test.iloc[:,2:].select_dtypes('float').columns.tolist()
    
    #数据转化流,离散字段独热编码，连续字段标准化
    ct = ColumnTransformer([
    ('cat',preprocessing.OneHotEncoder(drop='if_binary'),category_cols),
    ('num',preprocessing.StandardScaler(),numeric_cols)])
    
    #划分训练集和测试集
    df = pd.concat([application_train_and_test.iloc[:,:2],pd.DataFrame(ct.fit_transform(application_train_and_test),columns=ct.get_feature_names_out())],axis=1)
    df = rename_columns(df)
    df_train = df[df['TARGET'].notnull()]
    df_test = df[df['TARGET'].isnull()].drop(columns='TARGET',axis=1)
    
    x_train = df_train.drop(columns='TARGET',axis=1)
    y_train = df_train['TARGET']
    
    #交叉验证
    cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=520)
    result = cross_validate(model,x_train,y_train,cv=cv,scoring='roc_auc')
    
    #如果test为True，直接完成预测并得出sunmission表格
    if test:
        clf = model
        clf.fit(x_train,y_train)
        prob = clf.predict_proba(df_test)[:,1].reshape(-1,1)
        Submissing(prob,file_name)
    
    return result['test_score'].mean()

# 训练集和测试集的处理

In [20]:
#缺失值填补
application_train['DAYS_EMPLOYED'].fillna(0,inplace=True)
application_test['DAYS_EMPLOYED'].fillna(0,inplace=True)

application_train['OWN_CAR_AGE'].fillna(0,inplace=True)
application_test['OWN_CAR_AGE'].fillna(0,inplace=True)

In [21]:
#训练集性别里包含XNA没有特殊性，并且数据量少，直接删除
application_train = application_train[application_train['CODE_GENDER']!='XNA']

In [22]:
#处理异常值
application_train['DAYS_EMPLOYED_ANOM'] = application_train["DAYS_EMPLOYED"] == 365243
application_train["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

application_test['DAYS_EMPLOYED_ANOM'] = application_test["DAYS_EMPLOYED"] == 365243
application_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

application_train["DAYS_LAST_PHONE_CHANGE"].replace({0: np.nan}, inplace = True)

In [23]:
#月收入
application_train['AMT_INCOME_MONTH'] = application_train[['AMT_INCOME_TOTAL']]/12
application_test['AMT_INCOME_MONTH'] = application_test[['AMT_INCOME_TOTAL']]/12

In [24]:
#每月还款后剩多少钱
application_train['leave_money'] = application_train['AMT_INCOME_MONTH'] - application_train['AMT_ANNUITY']
application_test['leave_money'] = application_test['AMT_INCOME_MONTH'] - application_test['AMT_ANNUITY']

In [25]:
application_train['DAYS_EMPLOYED_PERC'] = application_train['DAYS_EMPLOYED'] / application_train['DAYS_BIRTH']
application_train['INCOME_CREDIT_PERC'] = application_train['AMT_INCOME_TOTAL'] / application_train['AMT_CREDIT']
application_train['INCOME_PER_PERSON'] = application_train['AMT_INCOME_TOTAL'] / application_train['CNT_FAM_MEMBERS']
application_train['ANNUITY_INCOME_PERC'] = application_train['AMT_ANNUITY'] / application_train['AMT_INCOME_TOTAL']
application_train['PAYMENT_RATE'] = application_train['AMT_ANNUITY'] / application_train['AMT_CREDIT']
# application_train['AIT_CC'] = application_train['AMT_INCOME_TOTAL'] / (application_train['CNT_CHILDREN']+1)

application_test['DAYS_EMPLOYED_PERC'] = application_test['DAYS_EMPLOYED'] / application_test['DAYS_BIRTH']
application_test['INCOME_CREDIT_PERC'] = application_test['AMT_INCOME_TOTAL'] / application_test['AMT_CREDIT']
application_test['INCOME_PER_PERSON'] = application_test['AMT_INCOME_TOTAL'] / application_test['CNT_FAM_MEMBERS']
application_test['ANNUITY_INCOME_PERC'] = application_test['AMT_ANNUITY'] / application_test['AMT_INCOME_TOTAL']
application_test['PAYMENT_RATE'] = application_test['AMT_ANNUITY'] / application_test['AMT_CREDIT']
# application_test['AIT_CC'] = application_test['AMT_INCOME_TOTAL'] / (application_test['CNT_CHILDREN']+1)

In [26]:
#将训练集和测试集合并，进行独热编码和哑变量处理
application_train_and_test = pd.concat([application_train,application_test],axis=0)
application_train_and_test.reset_index(drop=True,inplace=True)
application_train_and_test.shape

(356251, 130)

In [27]:
Test_score_function(application_train_and_test,LGBMClassifier(),file_name='2')

0.7637698219949149

# 处理previous_application

In [28]:
previous_application_CONTRACT_TYPE = previous_application[['SK_ID_CURR','NAME_CONTRACT_TYPE']]
previous_application_CONTRACT_TYPE

,SK_ID_CURR,NAME_CONTRACT_TYPE
0,271877,Consumer loans
1,108129,Cash loans
2,122040,Cash loans
3,176158,Cash loans
4,202054,Cash loans
...,...,...
1670209,352015,Consumer loans
1670210,334635,Consumer loans
1670211,249544,Consumer loans
1670212,400317,Cash loans


In [29]:
#Consumer loans与其他几种方式分开，划分一个新列
previous_application_CONTRACT_TYPE['Consumer_loans_CONTRACT_TYPE'] = previous_application_CONTRACT_TYPE['NAME_CONTRACT_TYPE'] == 'Consumer loans'
previous_application_CONTRACT_TYPE.rename(columns={'NAME_CONTRACT_TYPE':'pre_apl_NAME_CONTRACT_TYPE'},inplace=True)

#将previous_application_CONTRACT_TYPE进行独热编码
previous_application_CONTRACT_TYPE = OneHot_data(previous_application_CONTRACT_TYPE,True)[0]

In [30]:
#衍生的特征放入主数据，用测试函数测试
application_train_and_test1 = pd.merge(application_train_and_test,previous_application_CONTRACT_TYPE.groupby(by='SK_ID_CURR').mean()
                                       ,how='left',on='SK_ID_CURR')

In [50]:
Test_score_function(application_train_and_test1,LGBMClassifier(random_state=520),file_name='3')

0.7658681588511999

In [51]:
#处理异常值
previous_application['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_FIRST_DRAWING_ANON'] = previous_application["DAYS_FIRST_DRAWING"] == 365243

previous_application['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_FIRST_DUE_ANON'] = previous_application["DAYS_FIRST_DUE"] == 365243

previous_application['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_LAST_DUE_1ST_VERSION_ANON'] = previous_application["DAYS_LAST_DUE_1ST_VERSION"] == 365243

previous_application['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_LAST_DUE_ANON'] = previous_application["DAYS_LAST_DUE"] == 365243

previous_application['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_TERMINATION_ANON'] = previous_application["DAYS_TERMINATION"] == 365243

In [52]:
#计算月利率
previous_application['MONTH_RATIO'] = (previous_application['AMT_ANNUITY'] - (previous_application['AMT_CREDIT']/previous_application['CNT_PAYMENT']))/(previous_application['AMT_CREDIT']/previous_application['CNT_PAYMENT'])

In [53]:
#计算年利率
previous_application['YEAR_RATIO'] = previous_application['MONTH_RATIO'] * 12

In [54]:
#申请贷款和批准贷款之比
previous_application['APP_CREDIT_PERC'] = previous_application['AMT_APPLICATION'] / (previous_application['AMT_CREDIT']+1)

In [55]:
#购买商品的价格占首付金额比例
previous_application['AMT_GOODS_PRICEandDOWN_PAYMENT'] = previous_application['AMT_GOODS_PRICE'] / (previous_application['AMT_DOWN_PAYMENT']+1)
#购买商品的价格占批准贷款金额
previous_application['AMT_GOODS_PRICEandCREDIT'] = previous_application['AMT_GOODS_PRICE'] / (previous_application['AMT_CREDIT']+1)

In [56]:
#货款的基本利率加上客户获得的利率优惠比率
previous_application['RATE_INTEREST_PRIMARY_PRIVILEGED'] = previous_application['RATE_DOWN_PAYMENT'] + previous_application['RATE_INTEREST_PRIVILEGED']
#货款的基本利率比上客户获得的利率优惠比率
previous_application['RATE_INTEREST_PRIMARY_device_PRIVILEGED'] = previous_application['RATE_DOWN_PAYMENT'] + previous_application['RATE_INTEREST_PRIVILEGED']

In [57]:
#提取previous_application表中我认为有用的特征
previous_application_select = previous_application[['SK_ID_CURR','AMT_ANNUITY','CNT_PAYMENT','AMT_CREDIT','AMT_APPLICATION','AMT_DOWN_PAYMENT','AMT_GOODS_PRICE','DAYS_FIRST_DRAWING'
                     ,'DAYS_FIRST_DRAWING_ANON','DAYS_FIRST_DUE','DAYS_FIRST_DUE_ANON','DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE_1ST_VERSION_ANON'
                     ,'DAYS_LAST_DUE','DAYS_LAST_DUE_ANON','DAYS_TERMINATION','DAYS_TERMINATION_ANON','YEAR_RATIO','APP_CREDIT_PERC','AMT_GOODS_PRICEandDOWN_PAYMENT'
                                                    ,'AMT_GOODS_PRICEandCREDIT','RATE_INTEREST_PRIMARY_PRIVILEGED','RATE_INTEREST_PRIMARY_device_PRIVILEGED']]

previous_application_main = OneHot_data(previous_application_select,True)[0]   #独热编码

In [58]:
previous_application_main = previous_application_main.groupby(by='SK_ID_CURR',as_index=False).agg(['median','mean','sum','count'])
previous_application_main.columns = pd.Index([e[0] + "_" + e[1].upper() for e in previous_application_main.columns.tolist()])

In [59]:
#先分组后取均值，试试效果
Test_score_function(pd.merge(application_train_and_test1,previous_application_main
                                       ,how='left',on='SK_ID_CURR'),LGBMClassifier(random_state=520),file_name='s8',test=False)

0.7726603439800059

In [60]:
#模型有提升，保存
application_train_and_test2 = pd.merge(application_train_and_test1,previous_application_main.groupby(by='SK_ID_CURR').mean()
                                      ,how='left',on='SK_ID_CURR')

In [61]:
application_train_and_test2.shape

(356251, 223)

# bureau_balance 和 bureau 数据处理

In [28]:
#先将bureau_balance进行独热编码
bureau_balance = OneHot_data(bureau_balance,True)[0]

In [29]:
#对bureau_balance的MONTHS_BALANCE列分组后取['sum','mean','max','min']
bb_agg = bureau_balance.groupby(by='SK_ID_BUREAU',as_index=False).agg({'MONTHS_BALANCE':['sum','mean','max','min']})
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])

In [30]:
bb_agg.rename(columns={'SK_ID_BUREAU_':'SK_ID_BUREAU'},inplace=True)

In [31]:
#将bb_agg与bureau通过SK_ID_BUREAU连接,并删除SK_ID_BUREAU列
bureau = pd.merge(bureau,bb_agg, how='left', on='SK_ID_BUREAU')
bureau.drop(columns='SK_ID_BUREAU',axis=1,inplace=True)

In [32]:
bureau['CREDIT_CURRENCY'].value_counts()

CREDIT_CURRENCY
currency 1    1715020
currency 2       1224
currency 3        174
currency 4         10
Name: count, dtype: int64

In [33]:
#将CREDIT_CURRENCY划为二分类，当借款人持有一种以上货币类型的账户，标记为1
bureau['CREDIT_CURRENCY_CLF'] = bureau['CREDIT_CURRENCY'] != 'currency 1'

In [34]:
#未偿还的信贷总额占贷款总额比例
bureau['AMT_CREDIT_BACKRITIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']

In [35]:
#每月还款额占信贷总额比例
bureau['AMT_CREDIT_SUMANDANNUITY'] = bureau['AMT_ANNUITY'] / (bureau['AMT_CREDIT_SUM']+1)

In [36]:
bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']

In [37]:
bureau.columns

Index(['SK_ID_CURR', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'DAYS_CREDIT',
       'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT',
       'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM',
       'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE',
       'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY',
       'MONTHS_BALANCE_SUM', 'MONTHS_BALANCE_MEAN', 'MONTHS_BALANCE_MAX',
       'MONTHS_BALANCE_MIN', 'CREDIT_CURRENCY_CLF', 'AMT_CREDIT_BACKRITIO',
       'AMT_CREDIT_SUMANDANNUITY', 'CREDIT_DURATION', 'ENDDATE_DIF'],
      dtype='object')

In [38]:
#将bureau独热编码
bureau_hot = OneHot_data(bureau[['SK_ID_CURR','CREDIT_CURRENCY','CREDIT_CURRENCY_CLF','DAYS_CREDIT'
                                ,'MONTHS_BALANCE_SUM','MONTHS_BALANCE_MEAN','MONTHS_BALANCE_MAX'
                                ,'MONTHS_BALANCE_MIN','AMT_ANNUITY'
                                ,'AMT_CREDIT_SUM_OVERDUE','AMT_CREDIT_MAX_OVERDUE','AMT_CREDIT_SUM','AMT_CREDIT_SUM_DEBT'
                                ,'CREDIT_TYPE','AMT_CREDIT_SUMANDANNUITY','CREDIT_DURATION','ENDDATE_DIF']],True)[0]

In [39]:
bureau_hot = bureau_hot.groupby(by='SK_ID_CURR',as_index=False).agg(['median','mean','sum','count'])
bureau_hot.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bureau_hot.columns.tolist()])

In [40]:
#先分组后取均值，试试效果
Test_score_function(pd.merge(application_train_and_test2,bureau_hot
                                       ,how='left',on='SK_ID_CURR'),LGBMClassifier(random_state=520),file_name='s8',test=False)

0.776629228300331

In [41]:
application_train_and_test3 = pd.merge(application_train_and_test2,bureau_hot,how='left',on='SK_ID_CURR')

# POS_CASH_balance数据处理

In [42]:
POS_CASH_balance.drop(columns='SK_ID_PREV',axis=1,inplace=True)
POS_CASH_balance.head()

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,182943,-31,48.0,45.0,Active,0,0
1,367990,-33,36.0,35.0,Active,0,0
2,397406,-32,12.0,9.0,Active,0,0
3,269225,-35,48.0,42.0,Active,0,0
4,334279,-35,36.0,35.0,Active,0,0


In [43]:
#预期分期付款已支付的期数
POS_CASH_balance['CNT_INSTALMENT_PAYED'] = POS_CASH_balance['CNT_INSTALMENT'] - POS_CASH_balance['CNT_INSTALMENT_FUTURE']

In [44]:
POS_CASH_balance.columns

Index(['SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD', 'SK_DPD_DEF',
       'CNT_INSTALMENT_PAYED'],
      dtype='object')

In [45]:
#独热编码
POS_CASH_balance_hot = OneHot_data(POS_CASH_balance[['SK_ID_CURR','MONTHS_BALANCE','CNT_INSTALMENT'
                                                    ,'SK_DPD','CNT_INSTALMENT_PAYED','CNT_INSTALMENT_FUTURE'
                                                    ,'SK_DPD_DEF']],True)[0]
POS_CASH_balance_hot = POS_CASH_balance_hot.groupby(by='SK_ID_CURR',as_index=False).agg(['max','min','sum','mean','count'])
POS_CASH_balance_hot.columns = pd.Index([e[0] + "_" + e[1].upper() for e in POS_CASH_balance_hot.columns.tolist()])

In [46]:
Test_score_function(pd.merge(application_train_and_test3,POS_CASH_balance_hot
                            ,how='left',on='SK_ID_CURR'),LGBMClassifier(random_state=520),file_name='s9',test=False)   #mean

0.7790956845410089

In [47]:
application_train_and_test4 = pd.merge(application_train_and_test3,POS_CASH_balance_hot,how='left',on='SK_ID_CURR')

# credit_card_balance数据处理

In [48]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [49]:
credit_card_balance.drop(columns='SK_ID_PREV',axis=1,inplace=True)

In [50]:
#修改名字
credit_card_balance.rename(columns={'MONTHS_BALANCE':'ccb_MONTHS_BALANCE'},inplace=True)

In [51]:
# 剩余欠款占授信额度比例
credit_card_balance['AMT_BALANCE_ratio'] = credit_card_balance['AMT_BALANCE'] / (credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']+1)

In [52]:
#上期实际还款占总还款的比例
credit_card_balance['AMT_PAYMENT_CURRENT_intotal'] = credit_card_balance['AMT_PAYMENT_CURRENT'] / (credit_card_balance['AMT_PAYMENT_TOTAL_CURRENT']+1)
# credit_card_balance['AMT_PAYMENT_CURRENT_intotal'].replace({np.inf:np.nan},inplace=True)   #无限值替换为nan

In [53]:
#上期最小还款额占实际还款额比例
credit_card_balance['AMT_INST_MIN_REGULARITY_ratio'] = credit_card_balance['AMT_INST_MIN_REGULARITY'] / (credit_card_balance['AMT_PAYMENT_CURRENT']+1)
# credit_card_balance['AMT_INST_MIN_REGULARITY_ratio'].replace({np.inf:np.nan},inplace=True)   #无限值替换为nan

In [54]:
credit_card_balance['PAYMENT_DIV_MIN'] = credit_card_balance['AMT_PAYMENT_CURRENT'] / (credit_card_balance['AMT_INST_MIN_REGULARITY']+1)
credit_card_balance['LATE_PAYMENT'] = credit_card_balance['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
credit_card_balance['DRAWING_LIMIT_RATIO'] = credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'] / (credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']+1)

In [55]:
#独热编码
credit_card_balance_hot = OneHot_data(credit_card_balance[['SK_ID_CURR','ccb_MONTHS_BALANCE','AMT_BALANCE_ratio','AMT_PAYMENT_CURRENT_intotal','AMT_INST_MIN_REGULARITY_ratio'
                                                          ,'AMT_BALANCE','AMT_PAYMENT_CURRENT','PAYMENT_DIV_MIN'
                                                          ,'LATE_PAYMENT','DRAWING_LIMIT_RATIO']],True)[0]
credit_card_balance_hot = credit_card_balance_hot.groupby(by='SK_ID_CURR',as_index=False).agg(['mean','max','min','sum'])
credit_card_balance_hot.columns = pd.Index([e[0] + "_" + e[1].upper() for e in credit_card_balance_hot.columns.tolist()])

In [56]:
Test_score_function(pd.merge(application_train_and_test4,credit_card_balance_hot
                            ,how='left',on='SK_ID_CURR'),LGBMClassifier(random_state=520),file_name='666',test=False)

0.7803000525763275

In [57]:
#保存数据
application_train_and_test5 = pd.merge(application_train_and_test4,credit_card_balance_hot,how='left',on='SK_ID_CURR')

# installments_payments数据处理

In [58]:
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [59]:
installments_payments.drop(columns='SK_ID_PREV',axis=1,inplace=True)

In [60]:
#以周为单位
installments_payments['WEEKS_INSTALMENT'] = installments_payments['DAYS_INSTALMENT'] / 7
#以月为单位
installments_payments['MONTH_INSTALMENT'] = installments_payments['DAYS_INSTALMENT'] / 30

In [61]:
#以周为单位
installments_payments['WEEKS_ENTRY_PAYMENT'] = installments_payments['DAYS_ENTRY_PAYMENT'] / 7
#以月为单位
installments_payments['MONTH_ENTRY_PAYMENT'] = installments_payments['DAYS_ENTRY_PAYMENT'] / 30

In [62]:
#实际还款金额占当期应还款金额比例
installments_payments['AMT_INSTALMENT_inratio'] = round(installments_payments['AMT_PAYMENT']/(installments_payments['AMT_INSTALMENT']+1e-10),3)
#未还款金额
installments_payments['CANT_AMT_INSTALMENT'] = installments_payments['AMT_INSTALMENT'] - installments_payments['AMT_PAYMENT']

In [63]:
installments_payments['PAID_OVER_AMOUNT'] = installments_payments['AMT_PAYMENT'] - installments_payments['AMT_INSTALMENT']
installments_payments['PAID_OVER'] = (installments_payments['PAID_OVER_AMOUNT'] > 0).astype(int)
installments_payments['DPD'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']
installments_payments['DPD'] = installments_payments['DPD'].apply(lambda x: 0 if x <= 0 else x)
installments_payments['DBD'] = installments_payments['DAYS_INSTALMENT'] - installments_payments['DAYS_ENTRY_PAYMENT']
installments_payments['DBD'] = installments_payments['DBD'].apply(lambda x: 0 if x <= 0 else x)
installments_payments['LATE_PAYMENT'] = installments_payments['DBD'].apply(lambda x: 1 if x > 0 else 0)
installments_payments['INSTALMENT_PAYMENT_RATIO'] = installments_payments['AMT_PAYMENT'] / (installments_payments['AMT_INSTALMENT']+1)
installments_payments['LATE_PAYMENT_RATIO'] = installments_payments.apply(lambda x: x['INSTALMENT_PAYMENT_RATIO'] if x['LATE_PAYMENT'] == 1 else 0, axis=1)
installments_payments['SIGNIFICANT_LATE_PAYMENT'] = installments_payments['LATE_PAYMENT_RATIO'].apply(lambda x: 1 if x > 0.05 else 0)
installments_payments['DPD_7'] = installments_payments['DPD'].apply(lambda x: 1 if x >= 7 else 0)
installments_payments['DPD_15'] = installments_payments['DPD'].apply(lambda x: 1 if x >= 15 else 0)

In [64]:
installments_payments_select = installments_payments.groupby(by='SK_ID_CURR',as_index=False).agg(['mean','max','min','sum','median'])
installments_payments_select.columns = pd.Index([e[0] + "_" + e[1].upper() for e in installments_payments_select.columns.tolist()])

In [65]:
#带入所有特征得到的分数
Test_score_function(pd.merge(application_train_and_test5,installments_payments_select
                            ,how='left',on='SK_ID_CURR'),LGBMClassifier(random_state=520),file_name='666',test=False)   #'mean','max','min','sum','median'

0.7834023910236704

In [66]:
application_train_and_test6 = pd.merge(application_train_and_test5,installments_payments_select,how='left',on='SK_ID_CURR')

In [67]:
#保存数据
application_train_and_test6.to_csv('/root/autodl-tmp/now/usefuldata/application_train_and_test6.csv',index=False)

In [145]:
#带入所有特征得到的分数
Test_score_function(application_train_and_test6,LGBMClassifier(random_state=520),file_name='666',test=False)   #'mean','max','min','sum','median'

0.783696773859913

# 目标编码

In [72]:
 #选出离散特征和连续特征
category_cols = application_train_and_test6.iloc[:,2:].select_dtypes('object').columns.tolist() + application_train_and_test6.iloc[:,2:].select_dtypes('bool').columns.tolist()
numeric_cols = application_train_and_test6.iloc[:,2:].select_dtypes('int').columns.tolist() + application_train_and_test6.iloc[:,2:].select_dtypes('float').columns.tolist()

In [73]:
#划分出训练集和测试集
data_train = application_train_and_test6[application_train_and_test6['TARGET'].notnull()]
data_test = application_train_and_test6[application_train_and_test6['TARGET'].isnull()].drop(columns='TARGET',axis=1)

In [74]:
data_train.shape,data_test.shape

((307507, 531), (48744, 530))

In [75]:
for i in category_cols:
    print(f'{i}:',data_train[i].unique())

NAME_CONTRACT_TYPE: ['Cash loans' 'Revolving loans']
CODE_GENDER: ['M' 'F']
FLAG_OWN_CAR: ['N' 'Y']
FLAG_OWN_REALTY: ['Y' 'N']
NAME_TYPE_SUITE: ['Unaccompanied' 'Family' 'Spouse, partner' 'Children' 'Other_A' nan
 'Other_B' 'Group of people']
NAME_INCOME_TYPE: ['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']
NAME_EDUCATION_TYPE: ['Secondary / secondary special' 'Higher education' 'Incomplete higher'
 'Lower secondary' 'Academic degree']
NAME_FAMILY_STATUS: ['Single / not married' 'Married' 'Civil marriage' 'Widow' 'Separated'
 'Unknown']
NAME_HOUSING_TYPE: ['House / apartment' 'Rented apartment' 'With parents'
 'Municipal apartment' 'Office apartment' 'Co-op apartment']
OCCUPATION_TYPE: ['Laborers' 'Core staff' 'Accountants' 'Managers' nan 'Drivers'
 'Sales staff' 'Cleaning staff' 'Cooking staff' 'Private service staff'
 'Medicine staff' 'Security staff' 'High skill tech staff'
 'Waiters/barmen staff' 'Low-skill Labo

In [76]:
for i in category_cols:
    print(f'{i}:',data_test[i].unique())

NAME_CONTRACT_TYPE: ['Cash loans' 'Revolving loans']
CODE_GENDER: ['F' 'M']
FLAG_OWN_CAR: ['N' 'Y']
FLAG_OWN_REALTY: ['Y' 'N']
NAME_TYPE_SUITE: ['Unaccompanied' nan 'Family' 'Spouse, partner' 'Group of people'
 'Other_B' 'Children' 'Other_A']
NAME_INCOME_TYPE: ['Working' 'State servant' 'Pensioner' 'Commercial associate'
 'Businessman' 'Student' 'Unemployed']
NAME_EDUCATION_TYPE: ['Higher education' 'Secondary / secondary special' 'Incomplete higher'
 'Lower secondary' 'Academic degree']
NAME_FAMILY_STATUS: ['Married' 'Single / not married' 'Civil marriage' 'Widow' 'Separated']
NAME_HOUSING_TYPE: ['House / apartment' 'With parents' 'Rented apartment'
 'Municipal apartment' 'Office apartment' 'Co-op apartment']
OCCUPATION_TYPE: [nan 'Low-skill Laborers' 'Drivers' 'Sales staff' 'High skill tech staff'
 'Core staff' 'Laborers' 'Managers' 'Accountants' 'Medicine staff'
 'Security staff' 'Private service staff' 'Secretaries' 'Cleaning staff'
 'Cooking staff' 'HR staff' 'Waiters/barmen staff

In [142]:
Box_of_FONDKAPREMONT_MODE = data_train[['TARGET']].groupby(by=data_train['FONDKAPREMONT_MODE']).mean().reset_index()    #离散字段按照标签分组后取均值
kb = preprocessing.KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='kmeans')     #分箱
Box_of_FONDKAPREMONT_MODE['FONDKAPREMONT_MODE_KB'] = pd.DataFrame(kb.fit_transform(Box_of_FONDKAPREMONT_MODE[['TARGET']]).astype('int'))
Box_of_FONDKAPREMONT_MODE.drop(columns='TARGET',axis=1,inplace=True)

In [144]:
#二分类
Test_score_function(pd.merge(application_train_and_test6,Box_of_FONDKAPREMONT_MODE,how='left',on='FONDKAPREMONT_MODE'),LGBMClassifier(random_state=520),file_name='1')

0.7837525142620574

In [146]:
application_train_and_test7 = pd.merge(application_train_and_test6,Box_of_FONDKAPREMONT_MODE,how='left',on='FONDKAPREMONT_MODE')

In [147]:
#保存数据
application_train_and_test7.to_csv('/root/autodl-tmp/now/usefuldata/train_and_test7.csv',index=False)

In [150]:
application_train_and_test7.shape

(356251, 532)

# 模型训练和调参

In [7]:
application_train_and_test7 = pd.read_csv('/root/autodl-tmp/now/usefuldata/train_and_test7.csv')

In [8]:
 #选出离散特征和连续特征
category_cols = application_train_and_test7.iloc[:,2:].select_dtypes('object').columns.tolist() + application_train_and_test7.iloc[:,2:].select_dtypes('bool').columns.tolist()
numeric_cols = application_train_and_test7.iloc[:,2:].select_dtypes('int').columns.tolist() + application_train_and_test7.iloc[:,2:].select_dtypes('float').columns.tolist()

In [9]:
oh = preprocessing.OneHotEncoder(drop='if_binary')
oh.fit_transform(application_train_and_test7[category_cols])
hot_data = pd.DataFrame(oh.fit_transform(application_train_and_test7[category_cols]).toarray(),columns=cate_colName(oh,category_cols))  #衍生出的独热编码数据
hot_data = rename_columns(hot_data)
application_train_and_test7_temp = pd.concat([application_train_and_test7.drop(columns=application_train_and_test7[category_cols],axis=1),hot_data],axis=1)

In [10]:
application_train_and_test7_temp.shape

(356251, 657)

In [11]:
#进行数据归一化
scaler = preprocessing.StandardScaler()
application_train_and_test7_temp[numeric_cols] = scaler.fit_transform(application_train_and_test7_temp[numeric_cols])

In [12]:
#划分训练集和测试集
df_train = application_train_and_test7_temp[application_train_and_test7_temp['TARGET'].notnull()]
df_test = application_train_and_test7_temp[application_train_and_test7_temp['TARGET'].isnull()].drop(columns='TARGET',axis=1)

In [13]:
df_train.shape,df_test.shape

((307507, 657), (48744, 656))

In [14]:
x_train = df_train.drop(columns='TARGET',axis=1)
y_train = df_train['TARGET']

In [19]:
model = LGBMClassifier(num_leaves=74,n_estimators=2874,learning_rate=0.008531746815380674
                      ,colsample_bytree=0.19438015729100788,max_depth=19,reg_lambda=0.15260315601753538
                      ,subsample=0.1538823669977421,random_state=2023)
model.fit(x_train,y_train)   #0.79068  random_state=2023

LGBMClassifier(colsample_bytree=0.19438015729100788,
               learning_rate=0.008531746815380674, max_depth=19,
               n_estimators=2874, num_leaves=74, random_state=2023,
               reg_lambda=0.15260315601753538, subsample=0.1538823669977421)

In [20]:
proba = model.predict_proba(df_test)[:,1].reshape(-1,1)
Submissing(proba,file_name='best_score')